In [36]:
!pip install deep-translator 'transformers>=4.50.0' pillow 'torch>=2.3' accelerate pandas

Python(20338) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 792.2 kB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 1.7 MB/s eta 0:00:001.7 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
content = json.load(open('places.json'))

In [3]:
for c in content:
    if c['food_desc']:
        print(c['food_desc'])
    if c['img']:
        print(c['img'])

Cone de pastel, recheado com escabeche de carne de sol desfiada e maionese aioli. Acompanha molho da casa.
https://cdb-static-files.s3.amazonaws.com/wp-content/uploads/2025/03/14005511/1960-bar.jpg
Costela defumada com molho chilli estilo Tex-Mex, servida com chips de pastel crocante. Acompanha molho de pimenta da casa.
https://cdb-static-files.s3.amazonaws.com/wp-content/uploads/2025/03/15141933/24-do-4.jpg
Bolinho de mandioca e carne-seca com molho à base de abacaxi com rapadura.
https://cdb-static-files.s3.amazonaws.com/wp-content/uploads/2025/03/25111208/611-house-bar-bolinho-de-mani-montes-claros-mg-2024-daniel-maia-1.jpg
Filé de peixe temperado e marinado, servido com legumes frescos, ovos de codorna e coentro fresco, tudo cozido em um suave molho de tomate. Acompanha pirão feito com o caldo da peixada e arroz parabolizado temperado com alho, cebola e açafrão-da-terra.
https://cdb-static-files.s3.amazonaws.com/wp-content/uploads/2025/03/19115838/a-bodega-cachacaria_peixadinha-da-

In [5]:
# pip install accelerate

from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import requests
import torch
from PIL import Image

model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
mps_device = torch.device("mps")

def guess_restriction(food, restriction, image):

    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": f"You should evaluate using image and description of food if its {restriction}. Just answer 'yes' or 'no' if this food is {restriction}"}]
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": food}
            ]
        }
    ]


    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(mps_device)
    
    
    input_len = inputs["input_ids"].shape[-1]
    
    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=10, do_sample=False)
        generation = generation[0][input_len:]
    
    decoded = processor.decode(generation, skip_special_tokens=True)
    print(restriction, food, decoded)
    return decoded

# **Overall Impression:** The image is a close-up shot of a vibrant garden scene, 
# focusing on a cluster of pink cosmos flowers and a busy bumblebee. 
# It has a slightly soft, natural feel, likely captured in daylight.

In [8]:
food_restrictions = {'food_desc': [], 'vegetarian': [], 'vegan': [], 'gluten-free': []}
for c in content:
    if c['food_desc']:
        food = c['food_desc']
        image = c['img']
        food_restrictions['food_desc'].append(food)
        # vegetarian
        food_restrictions['vegetarian'].append(guess_restriction(food, 'vegetarian (no meat, fish and pig)', image))
        # vegan
        food_restrictions['vegan'].append(guess_restriction(food, 'vegan (no meat, fish, pig, eggs and other animal issues)', image))
        # gluten-free
        food_restrictions['gluten-free'].append(guess_restriction(food, 'gluten-free (no flour)', image))

Cone de pastel, recheado com escabeche de carne de sol desfiada e maionese aioli. Acompanha molho da casa.
vegetarian (no meat, fish and pig) Cone de pastel, recheado com escabeche de carne de sol desfiada e maionese aioli. Acompanha molho da casa. no
vegan (no meat, fish, pig, eggs and other animal issues) Cone de pastel, recheado com escabeche de carne de sol desfiada e maionese aioli. Acompanha molho da casa. No
gluten-free (no flour) Cone de pastel, recheado com escabeche de carne de sol desfiada e maionese aioli. Acompanha molho da casa. No
Costela defumada com molho chilli estilo Tex-Mex, servida com chips de pastel crocante. Acompanha molho de pimenta da casa.
vegetarian (no meat, fish and pig) Costela defumada com molho chilli estilo Tex-Mex, servida com chips de pastel crocante. Acompanha molho de pimenta da casa. No
vegan (no meat, fish, pig, eggs and other animal issues) Costela defumada com molho chilli estilo Tex-Mex, servida com chips de pastel crocante. Acompanha molho d

In [12]:
for k, f in enumerate(food_restrictions['vegetarian']):
    if 'yes' in f.lower():
        print(food_restrictions['food_desc'][k])

Bolinho vegano de mandioca com recheio de abobrinha e cenoura. Acompanha molho agridoce apimentado.
Bolinho de feijão-fradinho recheado com cogumelo. Acompanha: creme de avocado, vinagrete e molho de pimenta com dendê.
Fatias de pão Italiano, cobertos por uma passata de tomate, requeijão e provolone, finalizados com maçarico e folha de manjericão.
Quibe cru vegano com babaganush e purê de couve-flor. Servido no pão sírio.
Bolinho de mandioca recheado com queijo.
Bolinhos de alho-poró recheados com cream cheese do chef. Acompanha geleia de abacaxi picante.
Um disco de matrinxã na farinha de mandioca recheado com banana pacovã e castanha sapucaia.
Croquete de rabada com aioli de agrião e geleia de pimenta caseira.
Minipastéis de palmito, acompanha molho de maionese.
Dadinho de tapioca feito com mix de cogumelos, empanado e servido com geleia de pimenta e geleia de abacaxi.
Polpetas de berinjela com queijo parmesão, muçarela, manjericão e salsinha. Servidas com pasta de grão-de-bico.
Chip

In [22]:
len(food_restrictions['vegetarian'])

1061

In [13]:
len(content)

1063

In [26]:
food_restriction_labels = ['vegetarian', 'vegan', 'gluten-free']

for i, c in enumerate(content):
    for k, food in enumerate(food_restrictions['food_desc']):
        if c['food_desc'] and (food == c['food_desc']):
            for f in food_restriction_labels:
                if 'yes' in food_restrictions[f][k].lower():
                    content[i][f] = True
                else:
                    content[i][f] = False
            break
print(content)

[{'name': '1960 Bar', 'addrs': 'Rua 237, 307 | Setor Coimbra, Goiânia – GO  -  Quadra 45, Lote 01', 'geo': {'lat': -16.6810272, 'lng': -49.2825398}, 'url': 'https://comidadibuteco.com.br/buteco/1960-bar/', 'img': 'https://cdb-static-files.s3.amazonaws.com/wp-content/uploads/2025/03/14005511/1960-bar.jpg', 'food': 'Cone de Prata', 'food_desc': 'Cone de pastel, recheado com escabeche de carne de sol desfiada e maionese aioli. Acompanha molho da casa.', 'contact': '6292362753', 'work_hours': 'Segunda, das 17h às 22h | Terça, das 17h às 22h | Quinta, das 17h às 22h | Quarta, das 17h às 22h | Sexta, das 17h às 22h | Sábado, das 11h30 às 16h |', 'vegetarian': False, 'vegan': False, 'gluten-free': False}, {'name': '24 Do 4', 'addrs': 'Dr. Thomás, 151 | Nossa Sra das Graças, Manaus – AM  -   Loja 01', 'geo': {'lat': -3.0991763, 'lng': -60.01549209999999}, 'url': 'https://comidadibuteco.com.br/buteco/24-do-4/', 'img': 'https://cdb-static-files.s3.amazonaws.com/wp-content/uploads/2025/03/1514193

In [49]:
with open('places.json', 'w') as f:
    json.dump(content, f)

In [13]:
import pandas

df = pandas.read_json('places.json')

In [19]:
df[df.vegetarian & df['gluten-free']].count()

name           12
addrs          12
geo            12
url            12
img            12
food           12
food_desc      12
contact        12
work_hours     12
vegetarian     12
vegan          12
gluten-free    12
dtype: int64